In [1]:
import findspark
findspark.init(spark_home='D:\spark')
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext

In [2]:
input_uri = "mongodb://localhost:27017/Bitcoin.data"
output_uri = "mongodb://localhost:27017/Bitcoin.data"

conf=SparkConf()
conf.set('spark.mongodb.input.uri', input_uri)
conf.set('spark.mongodb.output.uri', output_uri)
conf.set('spark.mongodb.input.sampleSize', 50000)

In [3]:
sc=SparkContext.getOrCreate(conf=conf)
SQLContext=SQLContext(sc)
df=SQLContext.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- price_usd: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [4]:
df.show(5)

+--------------------+------------------+--------------------+
|                 _id|         price_usd|           timestamp|
+--------------------+------------------+--------------------+
|[60042141e31d5586...| 35032.71043588458|2021-01-17 12:36:...|
|[60042141e31d5586...| 35055.99784257462|2021-01-17 12:36:...|
|[60042143e31d5586...| 35054.54889815735|2021-01-17 12:36:...|
|[60042149e31d5586...|35057.570382921054|2021-01-17 12:36:...|
|[6004214fe31d5586...|  35073.6659496405|2021-01-17 12:36:...|
+--------------------+------------------+--------------------+
only showing top 5 rows



In [5]:
df.select("timestamp","price_usd").show(5)

+--------------------+------------------+
|           timestamp|         price_usd|
+--------------------+------------------+
|2021-01-17 12:36:...| 35032.71043588458|
|2021-01-17 12:36:...| 35055.99784257462|
|2021-01-17 12:36:...| 35054.54889815735|
|2021-01-17 12:36:...|35057.570382921054|
|2021-01-17 12:36:...|  35073.6659496405|
+--------------------+------------------+
only showing top 5 rows



In [6]:
# df1 = df.select("*").toPandas()
# cc = df1.set_index('timestamp')
# cc["price_usd"].plot()

In [7]:
df.count()

831

In [8]:
df.createOrReplaceTempView("BitcoinPrice")

In [9]:
SQLContext.sql("SELECT timestamp as ds, price_usd as y FROM BitcoinPrice ORDER BY ds").show(10)

+--------------------+------------------+
|                  ds|                 y|
+--------------------+------------------+
|2021-01-17 12:36:...| 35032.71043588458|
|2021-01-17 12:36:...| 35055.99784257462|
|2021-01-17 12:36:...| 35054.54889815735|
|2021-01-17 12:36:...|35057.570382921054|
|2021-01-17 12:36:...|  35073.6659496405|
|2021-01-17 12:36:...| 35068.84947610575|
|2021-01-17 12:36:...| 35079.08755231992|
|2021-01-17 12:37:...| 35107.52083601305|
|2021-01-17 12:37:...|35119.238905161365|
|2021-01-17 12:37:...|35106.350714309396|
+--------------------+------------------+
only showing top 10 rows



In [12]:
df_sql = SQLContext.sql("SELECT timestamp as ds, price_usd as y FROM BitcoinPrice ORDER BY ds")
df_sql = df_sql.select("*").toPandas()

In [16]:
df_sql = df_sql[:100]

In [11]:
df.explain()

== Physical Plan ==
*(1) Scan MongoRelation(MongoRDD[0] at RDD at MongoRDD.scala:51,Some(StructType(StructField(_id,StructType(StructField(oid,StringType,true)),true), StructField(price_usd,DoubleType,true), StructField(timestamp,TimestampType,true)))) [_id#0,price_usd#1,timestamp#2] PushedFilters: [], ReadSchema: struct<_id:struct<oid:string>,price_usd:double,timestamp:timestamp>


In [12]:
from pyspark.sql.types import *

result_shema = StructType([
    StructField("ds", TimestampType()),
    StructField("y", DoubleType()),
    StructField("yhat", DoubleType()),
    StructField("yhat_upper", DoubleType()),
    StructField("yhat_lower", DoubleType()),
])

In [18]:
import numpy as np

def percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/ y_true)) * 100

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation

model = Prophet(growth='linear',seasonality_mode='multiplicative', mcmc_samples=0, 
                yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)

model.fit(df_sql)
    
future_pd = model.make_future_dataframe(periods=10, freq='T')
forecast_pd = model.predict(future_pd)

f_pd = forecast_pd[['ds', 'yhat', 'yhat_upper', 'yhat_lower']].set_index('ds')
b_pd = Bitcoin_data[['ds', 'y']].set_index('ds')

results_pd = f_pd.join(b_pd, how='left')
results_pd.reset_index(level=0, inplace=True)

results_pd[['ds', 'y', 'yhat', 'yhat_upper', 'yhat_lower']]